<h2>  Part Three: Classifier training and performance assessment. </h2>	

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib
import skimage
from skimage import data
from skimage import io
from skimage import color
from skimage import feature
from skimage.transform import resize
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import sklearn
import os
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

<h3>  Train models using all of the following methods below. Be sure to drop the actual image column, and the encoding</h3>	Take note of the differences in accuracy, and methods.


## Cross validation scores

Logistic Regression

In [2]:
def logistic_regression_cv_grid_search(X_train_scale, y_train):
    C_array = [1, 10, 50]
    print('L2 regularizor')
    for C in C_array:
        clf = LogisticRegression(penalty='l2',C = C, multi_class = 'multinomial', solver='lbfgs',
                                 max_iter=1000)
        scores = cross_val_score(clf, X_train_scale, y_train, cv=5, scoring='accuracy')
        print('C = %f, score = %f' %(C, scores.mean()))
    
    print('L1 regularizor')
    for C in C_array:
        clf = LogisticRegression(penalty='l1',C = C, multi_class = 'multinomial', solver='saga', 
                                 max_iter=10000)
        scores = cross_val_score(clf, X_train_scale, y_train, cv=5, scoring='accuracy')
        print('C = %f, score = %f' %(C, scores.mean()))
    return

K-nearest Neighbors

In [3]:
def knn_cv_grid_search(X_train_scale, y_train):
    k_array = np.arange(1, 50, 2)
    
    for k in k_array:
        clf = KNeighborsClassifier(n_neighbors=k)
        scores = cross_val_score(clf, X_train_scale, y_train, cv=5, scoring='accuracy')
        print('K = %f, score = %f' %(k, scores.mean()))
    return

Classification Tree

In [4]:
def decision_tree_cv_grid_search(X_train_scale, y_train):
    max_depth_array = np.arange(1, 30, 2)
    
    criterion='gini'    
    print('loss function is %s' %criterion)
    for max_depth in max_depth_array:
        clf = DecisionTreeClassifier(criterion=criterion, max_depth=max_depth)
        scores = cross_val_score(clf, X_train_scale, y_train, cv=5, scoring='accuracy')
        print('max depth = %d, score = %f' %(max_depth, scores.mean()))
   
    criterion='entropy'    
    print('loss function is %s' %criterion)
    for max_depth in max_depth_array:
        clf = DecisionTreeClassifier(criterion=criterion, max_depth=max_depth)
        scores = cross_val_score(clf, X_train_scale, y_train, cv=5, scoring='accuracy')
        print('max depth = %d, score = %f' %(max_depth, scores.mean()))
    
    return

Random Forest

In [5]:
def random_forest_cv_grid_search(X_train_scale, y_train):
    max_samples_array = np.arange(0.1, 1, 0.2)
    max_features_array = np.arange(0.1, 1, 0.2)
    for n_estimators in [100, 200]:
        for max_depth in [7, 50, 100]:
            print('n_estimators = %d, max_depth = %d' %(n_estimators, max_depth))
            for max_samples in max_samples_array:
                for max_features in max_features_array:
                    clf = RandomForestClassifier(n_estimators = n_estimators, criterion='gini', max_depth=max_depth, 
                                                 bootstrap=True, n_jobs=-1, random_state=42, max_features=max_features, max_samples=max_samples)
                    scores = cross_val_score(clf, X_train_scale, y_train, cv=5, scoring='accuracy')
                    print('max_samples = %.2f, max_features = %.2f, score = %f' %(max_samples, max_features, scores.mean()))   
    
    return

Support Vector Machine

In [6]:
def svm_cv_grid_search(X_train_scale, y_train):
    C_array = [0.1, 1, 10, 50]
    gamma_array = [0.01, 0.1, 1, 10]
    
    print('kernal=linear')
    for C in C_array:
        clf = SVC(kernel='linear', C=C)
        scores = cross_val_score(clf, X_train_scale, y_train, cv=5, scoring='accuracy')
        print('C = %f, score = %f' %(C, scores.mean()))
    
    ####################
    print('kernal=rbf')
    for C in C_array:
        for gamma in gamma_array:
            clf = SVC(kernel='rbf', C=C, gamma=gamma)
            scores = cross_val_score(clf, X_train_scale, y_train, cv=5, scoring='accuracy')
            print('C=%.1f, gamma=%.2f, mean score = %f' %(C, gamma, scores.mean()))
    C=50
    gamma_array = np.arange(0.005, 0.025, 0.001)
    for gamma in gamma_array:
            clf = SVC(kernel='rbf', C=C, gamma=gamma)
            scores = cross_val_score(clf, X_train_scale, y_train, cv=5, scoring='accuracy')
            print('C=%.1f, gamma=%.3f, mean score = %f' %(C, gamma, scores.mean()))
  
    ###################
    print('kernal=poly')
    C_array = [0.1, 1, 10, 50]
    gamma_array = [0.01, 0.1, 1, 10]
    for C in C_array:
        for gamma in gamma_array:
            clf = SVC(kernel='poly', C=C, gamma=gamma)
            scores = cross_val_score(clf, X_train_scale, y_train, cv=5, scoring='accuracy')
            print('C=%.1f, gamma=%.3f, mean score = %f' %(C, gamma, scores.mean()))

    return

Read in feature data and preprocess feature

In [7]:
class_labels = ['Airplanes', 'Bear', 'Blimp', 'Comet', 'Crab', 'Dog', 'Dolphin', 'Giraffe', 'Goat', 'Gorilla', 'Kangaroo', 'Killer-Whale', 'Leopards', 'Llama', 'Penguin', 'Porcupine', 'Teddy-Bear', 'Triceratops', 'Unicorn', 'Zebra']
training_data = pd.read_pickle(os.path.join(os.getcwd(), 'training_feature_data.pkl'))
# drop highly correlated columns
columns_highly_corr = ['average_red', 'average_green', 'average_blue', 'value','dissimilarity','energy']
training_data = training_data.drop(columns=columns_highly_corr)

# These features are dropped because they are stupid
extra_columns_to_drop = ['image_size', 'aspect_ratio']
training_data = training_data.drop(columns=extra_columns_to_drop)

# preprocessing the data (shuffle, convert to numpy array for training and feature scaling)
training_data=training_data.sample(frac=1, random_state=42)
y_train = np.squeeze(training_data[['Encoding']].to_numpy())
X_train = training_data.drop(columns=['Encoding']).to_numpy()
X_train_scale = preprocessing.scale(X_train)

Run cross validations on learning set

In [8]:
# fine-tune hyperparameters using 5-fold cv
svm_cv_grid_search(X_train_scale, y_train)
logistic_regression_cv_grid_search(X_train_scale, y_train)
knn_cv_grid_search(X_train_scale, y_train)
decision_tree_cv_grid_search(X_train_scale, y_train)
random_forest_cv_grid_search(X_train_scale, y_train)    

kernal=linear
C = 0.100000, score = 0.368414
C = 1.000000, score = 0.374416
C = 10.000000, score = 0.367076
C = 50.000000, score = 0.365748
kernal=rbf
C=0.1, gamma=0.01, mean score = 0.198534
C=0.1, gamma=0.10, mean score = 0.195867
C=0.1, gamma=1.00, mean score = 0.093938
C=0.1, gamma=10.00, mean score = 0.093938
C=1.0, gamma=0.01, mean score = 0.352432
C=1.0, gamma=0.10, mean score = 0.375745
C=1.0, gamma=1.00, mean score = 0.151236
C=1.0, gamma=10.00, mean score = 0.113922
C=10.0, gamma=0.01, mean score = 0.387076
C=10.0, gamma=0.10, mean score = 0.359761
C=10.0, gamma=1.00, mean score = 0.156567
C=10.0, gamma=10.00, mean score = 0.116589
C=50.0, gamma=0.01, mean score = 0.389079
C=50.0, gamma=0.10, mean score = 0.364430
C=50.0, gamma=1.00, mean score = 0.156567
C=50.0, gamma=10.00, mean score = 0.116589
C=50.0, gamma=0.005, mean score = 0.389079
C=50.0, gamma=0.006, mean score = 0.385085
C=50.0, gamma=0.007, mean score = 0.387752
C=50.0, gamma=0.008, mean score = 0.392421
C=50.0, g

max_samples = 0.50, max_features = 0.90, score = 0.377068
max_samples = 0.70, max_features = 0.10, score = 0.369090
max_samples = 0.70, max_features = 0.30, score = 0.371763
max_samples = 0.70, max_features = 0.50, score = 0.377743
max_samples = 0.70, max_features = 0.70, score = 0.377081
max_samples = 0.70, max_features = 0.90, score = 0.375748
max_samples = 0.90, max_features = 0.10, score = 0.372414
max_samples = 0.90, max_features = 0.30, score = 0.381070
max_samples = 0.90, max_features = 0.50, score = 0.395063
max_samples = 0.90, max_features = 0.70, score = 0.378394
max_samples = 0.90, max_features = 0.90, score = 0.373085
n_estimators = 200, max_depth = 7
max_samples = 0.10, max_features = 0.10, score = 0.332447
max_samples = 0.10, max_features = 0.30, score = 0.350430
max_samples = 0.10, max_features = 0.50, score = 0.345765
max_samples = 0.10, max_features = 0.70, score = 0.348427
max_samples = 0.10, max_features = 0.90, score = 0.359745
max_samples = 0.30, max_features = 0.1

The model with best performance is SVM with rbf kernal, C = 50, gamma = 0.008 
Train this model on entire learning set

In [9]:
clf = SVC(kernel='rbf', C=50.0, gamma=0.008)
clf.fit(X_train_scale, y_train)
y_pred = clf.predict(X_train_scale)

Predict classes on test set and save the result

In [10]:
test_data=pd.read_pickle(os.path.join(os.getcwd(), 'test_feature_data.pkl'))
test_data = test_data.drop(columns=columns_highly_corr)
test_data = test_data.drop(columns=extra_columns_to_drop)
X_test = test_data.drop(columns=['Encoding']).to_numpy()
X_test_scale = preprocessing.scale(X_test)
y_pred = clf.predict(X_test_scale)
classification_test_data=pd.DataFrame()
classification_test_data['encoding']=y_pred
def encoding_to_class_name(encoding):
    class_labels = ['Airplanes', 'Bear', 'Blimp', 'Comet', 'Crab', 'Dog', 'Dolphin', 'Giraffe', 'Goat', 'Gorilla', 'Kangaroo', 'Killer-Whale', 'Leopards', 'Llama', 'Penguin', 'Porcupine', 'Teddy-Bear', 'Triceratops', 'Unicorn', 'Zebra']
    return class_labels[encoding]

classification_test_data['class_name']= classification_test_data['encoding'].apply(encoding_to_class_name)
classification_test_data = classification_test_data.drop(["encoding"],axis=1)
classification_test_data.to_csv(os.path.join(os.getcwd(), 'test_set_labels_pred.csv'), header=False, index=False)